In [1]:
import numpy as np
import pandas as pd
from pomegranate import *
import random
import os
import itertools
from functools import reduce

### Methods

In [34]:
def init(K,X):
    """This method initializes the models for EM
    
    K: Number of clusters
    X: Data
    
    Return: x_k, models,alpha_k, indices_array, CML
    """
    LENGTH, DIMENSION = X.shape
    models = []
    x_k = [[] for i in range(K)] #initialize K empty data arrays
    alpha_k = []
    indices_array = [[] for i in range(K)]
    print("**************** K =", K ,"************************")


    # Sequences for initial CL Multinet Estimation
    # Make K subsets of data
    for i in range(LENGTH):
        random_integer = random.randint(0,K-1)
        x_k[random_integer].append(X[i])
        indices_array[random_integer].append(i)

    for i in range(K):
        print("Length of model",i+1,":",len(x_k[i]))
        alpha_k.append(len(x_k[i])/LENGTH)
        model = BayesianNetwork.from_samples(x_k[i],algorithm='chow-liu') 
        models.append(model)

    print("Initial Model Structures",alpha_k)
    for model in models:
        print(model.structure)

    print("Initial Alphas",alpha_k)

    CML = 0

    for i in range(K):
        x = x_k[i]
        model = models[i]
        CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[i])
    print("Initial CML",CML)
    
    return x_k,models,alpha_k,indices_array,CML
           
def e_step(K,X,x_k,models,alpha_k,indices_array):
    """This method performs the E step in EM for the mth iteration
    
    K: Number of clusters
    X: Data
    x_k: Previously classified data in the (m-1)th step
    models: models from the (m-1)th step
    alpha_k: alphas from the previous step
    indices_array: current indices of the original data (for each cluster)
    
    Return: x_k,models, alpha_k,indices_array
    """
    x_k_temp = [[] for i in range(K)] #initialize K empty data arrays for C step (assign)
    indices_array = [[] for i in range(K)]

    #E Step: Calculate each point's posterior probability for K clusters (trees)
    for idx_first,x in enumerate(X):
        model_prob = []
        for idx,model in enumerate(models): # K trees
            try:
                model_prob.append(model.probability(x))
            except KeyError: #if a point doesn't exist in a tree, then the probability is zero
                model_prob.append(0)
        total = [a*b for a,b in zip(model_prob,alpha_k)]
        max_prob_idx = total.index(max(total)) #return index of the max posterior probability
        x_k_temp[max_prob_idx].append(x)
        indices_array[max_prob_idx].append(idx_first)

    #C step: Assign data-points to the trees that maximize their posterior probability
    x_k = x_k_temp
    alpha_k = [len(x_k[i])/LENGTH for i in range(K)]
    models = []
    for j in range(K):
        model = BayesianNetwork.from_samples(x_k[j],algorithm='chow-liu') 
        models.append(model)
        
    return x_k,models, alpha_k,indices_array

def m_step(K,x_k, models, alpha_k,CML):
    """This method performs the M step in EM for the mth iteration
    
    K: Number of clusters
    x_k: Previously classified data in the (m-1)th step
    models: models from the (m-1)th step
    alpha_k: alphas from the previous step
    CML: Classification Maximum Likelihood    
    
    Return: x_k,models, alpha_k,indices_array
    """
    #M step: Calculate the CML criterion and re-estimate parameters
    init_CML = CML
    CML=0
    for j in range(K):
        x = x_k[j]
        model = models[j]
        CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[j])
    return CML, models

# def s_step(K,X):

#     LENGTH, DIMENSION = X.shape
    
#     #S Step
#     x_k = [[] for i in range(K)] #initialize K empty data arrays
#     #S step: Assign data-points randomly
#     for j in range(LENGTH):
#         x_k[random.randint(0,K-1)].append(X[j])
#     alpha_k = [len(x_k[k])/LENGTH for k in range(K)]
#     models = []
#     for j in range(K):
#         model = BayesianNetwork.from_samples(x_k[j],algorithm='chow-liu') 
#         models.append(model)

#     #M step: Calculate the CML criterion and re-estimate parameters
#     init_CML = CML
#     CML=0
#     for j in range(K):
#         x = x_k[j]
#         model = models[j]
#         CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[j])
#     print("New CML is:", CML)   


def save(indices_array, path):
    """This method saves the clustered data
    
    indices_array: array of the indices (of the original data) for each cluster
    path: path to save to, sample: '/Users/akankshitadash/Desktop/Bayesian Networks1/RPF_chrE/'
    Directory should already exist, and contain subdirectories of Genes/ and AccNum/
    
    Return: x_k,models, alpha_k,indices_array
    """
    for idx,indices in enumerate(indices_array):
        genes=[]
        acc_nums=[]
        for index in indices:
            genes.append(df.iloc[index]['GeneName'])
            acc_nums.append(df.iloc[index]['AccNum'])
        print(len(indices),len(genes),len(acc_nums))
    #             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array)))
    #             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/Genes/')
    #             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/AccNums/')
        with open(path+str(len(indices_array))+'/Gene'+str(idx+1)+'.txt','w') as f:
            for gene in genes:
                f.write("%s\n" % gene)
        with open(path+str(len(indices_array))+'/AccNum'+str(idx+1)+'.txt','w') as f:
            for acc_num in acc_nums:
                f.write("%s\n" % acc_num)
    
def em(K,X,path):    
    """This method performs EM
    
    K: Number of clusters
    X: Discrete data
    path: path to save to
    Return: None
    """
    
    x_k,models,alpha_k,indices_array,CML = init(K,X) #initialize K models
    prev_CML = CML
    
    for i in range(100): #start with 100 iterations of EM
        x_k, models, alpha_k,indices_array = e_step(K,X,x_k,models,alpha_k,indices_array)
        CML, models = m_step(K,x_k, models, alpha_k,CML)
        if(prev_CML==CML):
            for model in models:
                print(model.structure)
            break
        else:
            prev_CML = CML
            print("CML is",CML)
    save(indices_array,path)

In [3]:
# df_rnaseq = pd.read_csv('AdjustedRPKMOutput/RNASeq_chrE.txt',sep='\t')
# df_rpf = pd.read_csv('AdjustedRPKMOutput/RPF_chrE.txt',sep='\t')
# df_rnaseq = df_rnaseq[(df_rnaseq['cdReads0'] >= 10) & (df_rnaseq['cdReads1'] >= 10) & (df_rnaseq['cdReads2'] >= 10)& (df_rnaseq['cdReads3'] >= 10)& (df_rnaseq['cdReads4'] >= 10)]
# df_rpf = df_rpf[(df_rpf['cdReads0'] >= 10) & (df_rpf['cdReads1'] >= 10) & (df_rpf['cdReads2'] >= 10)& (df_rpf['cdReads3'] >= 10)& (df_rpf['cdReads4'] >= 10)]
# df_TE = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [df_rpf,df_rnaseq])
# df_TE[['cdRPKM0_x']] = df_TE[['cdRPKM0_x']].div(df_TE['cdRPKM0_y'].values,axis=0)
# df_TE[['cdRPKM1_x']] = df_TE[['cdRPKM1_x']].div(df_TE['cdRPKM1_y'].values,axis=0)
# df_TE[['cdRPKM2_x']] = df_TE[['cdRPKM2_x']].div(df_TE['cdRPKM2_y'].values,axis=0)
# df_TE[['cdRPKM3_x']] = df_TE[['cdRPKM3_x']].div(df_TE['cdRPKM3_y'].values,axis=0)
# df_TE[['cdRPKM4_x']] = df_TE[['cdRPKM4_x']].div(df_TE['cdRPKM4_y'].values,axis=0)
# for i in range(0,5):
#     df_TE.rename(columns={'cdRPKM'+str(i)+'_x':'TE'+str(i)}, inplace=True)
# df_TE = df_TE[['AccNum','GeneName','TE0','TE1','TE2','TE3','TE4']]
# df_TE.to_csv('AdjustedRPKMOutput/TE_chrE.txt',sep='\t')

### Read Data

In [4]:
path = 'Replicate2/TE_chrE_filtered.txt'
df = pd.read_csv(path,sep='\t')

In [5]:
df.head(5)

,AccNum,GeneName,TE0,TE1,TE2,TE3,TE4,foldTE1,foldTE2,foldTE3,foldTE4
0,NM_001003803,ATP5S,1.771663,1.623404,1.465394,0.647103,1.930694,-0.148259,-0.306269,-1.124560,0.159031
1,NM_001003800,BICD2,-0.606712,-0.590100,-0.680005,-0.491531,0.363802,0.016612,-0.073293,0.115181,0.970514
2,NM_016640,MRPS30,0.655493,0.615126,0.244848,0.162678,0.974128,-0.040367,-0.410645,-0.492815,0.318635
3,NM_001304467,EEF1AKMT2,0.139172,0.507027,0.150521,-0.456894,1.292490,0.367854,0.011348,-0.596067,1.153317
4,NM_001080825,TMEM120B,0.902133,1.213467,0.150521,0.884143,2.177961,0.311334,-0.751612,-0.017991,1.275828


In [6]:
df.keys()

Index(['AccNum', 'GeneName', 'TE0', 'TE1', 'TE2', 'TE3', 'TE4', 'foldTE1',
       'foldTE2', 'foldTE3', 'foldTE4'],
      dtype='object')

In [21]:
df[['TE0', 'TE1', 'TE2', 'TE3', 'TE4']] = df[['TE0', 'TE1', 'TE2', 'TE3', 'TE4']].apply(lambda x:2**x)

In [22]:
df.head()

,AccNum,GeneName,TE0,TE1,TE2,TE3,TE4,foldTE1,foldTE2,foldTE3,foldTE4
0,NM_001003803,ATP5S,3.414474,3.081011,2.761389,1.566021,3.812386,-0.148259,-0.306269,-1.124560,0.159031
1,NM_001003800,BICD2,0.656691,0.664297,0.624163,0.711270,1.286812,0.016612,-0.073293,0.115181,0.970514
2,NM_016640,MRPS30,1.575154,1.531692,1.184968,1.119363,1.964454,-0.040367,-0.410645,-0.492815,0.318635
3,NM_001304467,EEF1AKMT2,1.101273,1.421118,1.109970,0.728553,2.449504,0.367854,0.011348,-0.596067,1.153317
4,NM_001080825,TMEM120B,1.868827,2.318943,1.109970,1.845667,4.525136,0.311334,-0.751612,-0.017991,1.275828


In [23]:
X = df[['TE0', 'TE1', 'TE2', 'TE3', 'TE4', 'foldTE1',
       'foldTE2', 'foldTE3', 'foldTE4']].values.round()

In [24]:
X[:5]

array([[ 3.,  3.,  3.,  2.,  4., -0., -0., -1.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  0., -0.,  0.,  1.],
       [ 2.,  2.,  1.,  1.,  2., -0., -0., -0.,  0.],
       [ 1.,  1.,  1.,  1.,  2.,  0.,  0., -1.,  1.],
       [ 2.,  2.,  1.,  2.,  5.,  0., -1., -0.,  1.]])

In [25]:
print(X.shape)

(6175, 9)


In [26]:
LENGTH, DIMENSION = X.shape

In [27]:
print(np.min(X))

-3.0


In [28]:
print(np.max(X))

34.0


#### Digitize the data

In [29]:
bin_size = 100 #state number of bins here, multiple of 5
step = (np.max(X)-np.min(X))/bin_size
bins = np.arange(np.min(X),np.max(X)+0.1,step)
print(bins)
X = np.digitize(X,bins)

[-3.   -2.63 -2.26 -1.89 -1.52 -1.15 -0.78 -0.41 -0.04  0.33  0.7   1.07
  1.44  1.81  2.18  2.55  2.92  3.29  3.66  4.03  4.4   4.77  5.14  5.51
  5.88  6.25  6.62  6.99  7.36  7.73  8.1   8.47  8.84  9.21  9.58  9.95
 10.32 10.69 11.06 11.43 11.8  12.17 12.54 12.91 13.28 13.65 14.02 14.39
 14.76 15.13 15.5  15.87 16.24 16.61 16.98 17.35 17.72 18.09 18.46 18.83
 19.2  19.57 19.94 20.31 20.68 21.05 21.42 21.79 22.16 22.53 22.9  23.27
 23.64 24.01 24.38 24.75 25.12 25.49 25.86 26.23 26.6  26.97 27.34 27.71
 28.08 28.45 28.82 29.19 29.56 29.93 30.3  30.67 31.04 31.41 31.78 32.15
 32.52 32.89 33.26 33.63 34.  ]


In [30]:
print(X[:5])

[[17 17 17 14 19  9  9  6  9]
 [11 11 11 11 11  9  9  9 11]
 [14 14 11 11 14  9  9  9  9]
 [11 11 11 11 14  9  9  6 11]
 [14 14 11 14 22  9  6  9 11]]


### Sample network

In [31]:
model = BayesianNetwork.from_samples(X,algorithm='chow-liu')

In [32]:
model.structure

((), (0,), (1,), (4,), (1,), (6,), (7,), (8,), (0,))

### Perform EM

In [35]:
path = '/Users/akankshitadash/Desktop/Bayesian Networks/TE+log2foldTE/'
for k in range(4,7):
    em(k,X,path)

**************** K = 4 ************************
Length of model 1 : 1535
Length of model 2 : 1571
Length of model 3 : 1513
Length of model 4 : 1556
Initial Model Structures [0.248582995951417, 0.25441295546558707, 0.24502024291497976, 0.2519838056680162]
((), (0,), (1,), (4,), (1,), (6,), (7,), (8,), (0,))
((), (0,), (1,), (2,), (1,), (6,), (7,), (0,), (7,))
((), (0,), (1,), (4,), (1,), (6,), (7,), (0,), (7,))
((), (0,), (1,), (2,), (1,), (6,), (7,), (8,), (0,))
Initial Alphas [0.248582995951417, 0.25441295546558707, 0.24502024291497976, 0.2519838056680162]
Initial CML -55280.68807014222
CML is -48910.149876741576
CML is -47667.80246199622
CML is -47161.470397968165
CML is -46897.95363010376
CML is -46784.857705239614
CML is -46675.91701221362
CML is -46633.83843372024
CML is -46594.04826896972
CML is -46569.75373514529
CML is -46555.60338888462
CML is -46544.11493502906
CML is -46524.80300338027
CML is -46518.744634163515
((), (0,), (1,), (4,), (1,), (6,), (8,), (8,), (0,))
((), (0,),